# Bayesian Bandits: An Application of Thompson Sampling

#### EE 290S/CS 194: Machine Learning for Sequential Decision Making Under Uncertainty (Fall 2018)

#### Authors: Julian Chan, Daniel Ho, Bernie Wang

This notebook implements the Thompson Sampling algorithm described in [Analysis of Thompson Sampling for the Multi-Armed Bandit Problem](http://proceedings.mlr.press/v23/agrawal12/agrawal12.pdf) by Shipra Agrawal and Navin Goyal and applies it to the Bayesian Bandit problem.

** Note that "bandit" and "arm" are used interchangeably in this notebook (and in the Online Learning literature in general). **

## Table of Contents
[Section 1: Thompson Sampling for Bernoulli Bandits](#section1)

[Section 2: Thompson Sampling for General Stochastic Bandits](#section2)

## Section 1: Thompson Sampling for Bernoulli Bandits <a name="section1"></a>

We will first examine the Bernoulli Bandit case where:
* Rewards $r_t \in \{0, 1\}$ drawn from a Bernoulli distribution
* For arm $i$, the probability of success ($r_{t,i} = 1$) is its mean, $\mu_i$

We maintain a Bayesian prior on the arm means $\mu_i$. For Bernoulli rewards (either 0 or 1), it turns out that a Beta distribution is a convenient choice of priors. This is because if the prior is a Beta($\alpha$, $\beta$) distribution, then updating the posterior distributions become much simpler. After observing a Bernoulli trial, the posterior distribution is:
* Beta($\alpha+1$, $\beta$) if the trial succeeded (reward = 1)
* Beta($\alpha$, $\beta+1$) if the trial failed (reward = 0)

The Thompson Sampling algorithm initializes a uniform prior on all arms, meaning that arm $i$ has a prior Beta(1,1) on $\mu_i$ because Beta(1,1) is a uniform distribution on (0,1).

Define:
* $S_i(t)$ = number of successes (reward = 1) for arm $i$ up to time $t$
* $F_i(t)$ = number of failures (reward = 0) for arm $i$ up to time $t$

The algorithm will then update the distribution on $\mu_i$ as Beta($S_i(t)+1$, $F_i(t)+1$), sample from these posterior distributions, and play an arm according to the probability of its mean being the largest.

---
#### Algorithm 1: Thompson Sampling for Bernoulli Bandits
For each arm $i=1,...,N$, set $S_i(1)=0$, $F_i(1)=0$

For each  $t=1,2,...$ do

* For each arm $i=1,...,N$, sample $\theta_i(t)$ from the Beta($S_i(t)+1$, $F_i(t)+1$) distribution
* Play arm $i(t) := argmax_i$ $\theta_i(t)$ and observe reward $r_t$
* If $r_t=1$, then $S_i(t) = S_i(t)+1$, else $F_i(t) = F_i(t)+1$
---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import copy

Note: When constructing bandits below, create subclass of MultiArmedBandit to inherit class methods.

In [ ]:
class MultiArmedBandit(object):
    def __init__(self, num_bandits):
        """num_bandits must be at least 2"""
        
        assert num_bandits >= 2, "Number of bandits must be at least 2"
        
        # Initialize the means for each of the bandits randomly
        self.bandits = np.random.sample(num_bandits)
        self.num_bandits = num_bandits
        # Keep track of the largest probability to use in regret calculation as "best bandit"
        self.max_prob = np.max(self.bandits)
    
    # Should define this method when extending the class
    def chooseBandit(self, bandit_num):
        """
        Chooses the indicated bandit and returns True if we get a reward, False otherwise.
        """
        pass
    
    def computeRegret(self, bandit_num):
        """
        Computes the regret accrued by the indicated bandit. By definition, this is how much reward we lost by
        choosing the indicated bandit instead of the best one (indicated by the bandit with the largest mean).
        """
        # Index out of range
        assert bandit_num >= 0 and bandit_num < self.num_bandits, \
            "Index Out of Range: {}. There are {} bandits.".format(bandit_num, self.num_bandits)
        
        # Difference in expected reward for choosing a suboptimal bandit
        return self.max_prob - self.bandits[bandit_num]

    def __repr__(self):
        """Print out the means for each bandit arm."""
        return "Bandits: " + self.bandits.__str__()
    
    def __str__(self):
        return self.__repr__()

### Implement chooseBandit function. Returns True if bandit yields reward of 1, else return False.

In [ ]:
class BernoulliBandit(MultiArmedBandit):
    def __init__(self, num_bandits):
        super(BernoulliBandit, self).__init__(num_bandits)
        
    def chooseBandit(self, bandit_num):
        """
        Chooses the indicated bandit and returns True if we get a reward, False otherwise.
        """
        # Index out of range
        assert bandit_num >= 0 and bandit_num < self.num_bandits, \
            "Index Out of Range: {}. There are {} bandits.".format(bandit_num, self.num_bandits)
            
        # Sample from Bernoulli distribution with probability of success equal to the bandit mean
        raise NotImplementedError

In [ ]:
# CHANGE THIS TO EXPERIMENT
num_bandits = 10
num_iterations = 10000

TS_bandits = BernoulliBandit(num_bandits)
UCB_bandits = copy.deepcopy(TS_bandits)    # To ensure that they are instantiated with the same bandit means

### Implement Thompson Sampling and UCB Algorithms

In [ ]:
def draw_sample_means_TS(stats):
    """
    USED FOR THOMPSON SAMPLING.
    
    Samples the probability of each bandit yielding a reward given the number of successes and failures so far.
    This is computed using the Beta distribution.
    
    Parameters:
    stats = [{'num_wins': __, 'num_losses': __}, ...]
    len(stats) should be equal to the number of bandits
    
    Return:
    reward_probs: array of reward probability for each bandit
    """
    reward_probs = np.zeros(len(stats))
    raise NotImplementedError

def compute_heuristic_UCB(stats, curr_iteration=None):
    """
    USED FOR UCB.
    
    Computes the heuristic (average reward for bandit i) + sqrt(2*ln(n)/(# times bandit i is chosen))
    where n is the number of actions taken so far.
    
    Parameters:
    stats = [{'num_wins': __, 'num_losses': __}, ...]
    curr_iteration = number of iterations passed/actions taken so far
    len(stats) should be equal to the number of bandits
    
    Return:
    heuristics: array of UCB heuristic values
    """
    heuristics = np.zeros(len(stats))
    raise NotImplementedError

In [ ]:
def runSamplingAlgo(bandits, sampling_algo):
    """
    Parameters:
    bandits (MultiArmedBandit): instance of MultiArmedBandit class
    sampling_algo (str): sampling algorithm to use
    
    Return:
    round_stats: array of bandit chosen and regret at each iteration
    """
    # bandit_stats contains number of wins and losses for each bandit
    bandit_stats = np.zeros(num_bandits, dtype=[('num_wins', np.int), ('num_losses', np.int)])
    # round_stats contains bandit chosen and regret at each iteration
    round_stats = np.zeros(num_iterations, dtype=[('bandit', np.int), ('regret', np.float64)])
    
    for i in range(1, num_iterations):
        # Deep copy statistics from the previous iteration
        round_stats[i] = round_stats[i - 1].copy()
        # TODO: Compute the sampling heuristic for each bandit based on given algorithm
        
        # TODO: Choose the bandit with the maximum heuristic value
        
        # TODO: Update the round_stats and bandit_stats given selected bandit
        
    raise NotImplementedError

### Run both Thompson Sampling and UCB algorithm to get round_stats for each algorithm

In [ ]:
TS_round_stats = # Fill in
UCB_round_stats = # Fill in

### Plot round stats to compare the two algorithms

In [ ]:
# Plot the chosen bandit at each iteration
fig, ax = plt.subplots(1, 2, figsize=(13,5))
ax[0].scatter(range(num_iterations), TS_round_stats['bandit'], marker='x')
ax[0].set_yticks(range(num_bandits))
ax[0].set_xlabel("Iteration Number")
ax[0].set_ylabel("Selected Bandit")
ax[0].set_title("Bandit chosen at each iteration (Thompson Sampling)")

ax[1].scatter(range(num_iterations), UCB_round_stats['bandit'], marker='x')
ax[1].set_yticks(range(num_bandits))
ax[1].set_xlabel("Iteration Number")
ax[1].set_ylabel("Selected Bandit")
ax[1].set_title("Bandit chosen at each iteration (UCB)")

max_mean_index = np.argmax(TS_bandits.bandits)
for i in range(num_bandits):
    if i == max_mean_index:
        ax[1].text(x=num_iterations+700, y=i-0.2, s="$\mu_{}$ = {}".format(i, TS_bandits.bandits[i]), color="red")
    else:
        ax[1].text(x=num_iterations+700, y=i-0.2, s="$\mu_{}$ = {}".format(i, TS_bandits.bandits[i]))

In [ ]:
# Plot the accumulated regret through the experiment
fig, ax = plt.subplots(1, 2, figsize=(20,5))
ax[0].plot(range(num_iterations), TS_round_stats['regret'])
ax[0].set_xlabel("Iteration Number")
ax[0].set_ylabel("Cumulative Regret")
ax[0].set_title("Total Regret (Thompson Sampling)")

ax[1].plot(range(num_iterations), UCB_round_stats['regret'])
ax[1].set_xlabel("Iteration Number")
ax[1].set_ylabel("Cumulative Regret")
ax[1].set_title("Total Regret (UCB)")

## Section 2: Thompson Sampling for General Stochastic Bandits <a name="section2"></a>

Now we'll extend the Thompson Sampling algorithm to general stochastic bandit case, where:
* Rewards $r_i \in [0,1]$ drawn from an arbitrary unknown distribution
* Each arm $i$ has mean $\mu_i$

However, we will adapt the algorithm in a way that allows us to reuse the analysis of the Bernoulli Bandit. We will again make use of the Beta distribution as a convenient prior on the bandit arms.

The idea is that after observing the reward $\tilde{r}_t \in [0,1]$ at time $t$, we will perform a Bernoulli trial with probability of success $p = \tilde{r}_t$. Let the outcome of this Bernoulli trial be $r_t \in \{0,1\}$. Again, let $S_i(t)$ and $F_i(t)$ be the number of successes and failures up to time $t$, respectively, as before. 

Notice that the probability of observing a success in the Bernoulli trial after playing an arm $i$ in the new algorithm is the mean of that arm $\mu_i$. If we let $f_i$ denote the pdf of the unknown reward distribution for arm $i$. So, when playing arm $i$:
$$\Pr(r_t=1) = \int_{0}^{1} \tilde{r}f_i(\tilde{r})d\tilde{r} = \mu_i$$

---
#### Algorithm 2: Thompson Sampling for General Stochastic Bandits
For each arm $i=1,...,N$, set $S_i(1)=0$, $F_i(1)=0$

For each  $t=1,2,...$ do

* For each arm $i=1,...,N$, sample $\theta_i(t)$ from the Beta($S_i(t)+1$, $F_i(t)+1$) distribution
* Play arm $i(t) := argmax_i$ $\theta_i(t)$ and observe reward $\tilde{r}_t$
* Perform a Bernoulli trial with success probability $\tilde{r}_t$ and observe output $r_t$
* If $r_t=1$, then $S_i(t) = S_i(t)+1$, else $F_i(t) = F_i(t)+1$
---

### Implement chooseBandit function. Returns True if bandit yields reward 1, else return False.

In [ ]:
class UniformBandit(MultiArmedBandit):
    def __init__(self, num_bandits):
        super(UniformBandit, self).__init__(num_bandits)
        
    def chooseBandit(self, bandit_num):
        """
        Chooses the indicated bandit and returns True if we get a reward, False otherwise.
        """
        # Index out of range
        assert bandit_num >= 0 and bandit_num < self.num_bandits, \
            "Index Out of Range: {}. There are {} bandits.".format(bandit_num, self.num_bandits)
            
        # Sample from uniform distribution with probability of success equal to the bandit mean
        raise NotImplementedError

### Run sampling algorithm and plot as done before

In [ ]:
# CHANGE THIS TO EXPERIMENT
num_bandits = 10
num_iterations = 10000

TS_bandits = UniformBandit(num_bandits)
UCB_bandits = copy.deepcopy(TS_bandits)    # To ensure that they are instantiated with the same bandit means

In [ ]:
TS_round_stats = # Fill in
UCB_round_stats = # Fill in

In [ ]:
# Plot the chosen bandit at each iteration
fig, ax = plt.subplots(1, 2, figsize=(13,5))
ax[0].scatter(range(num_iterations), TS_round_stats['bandit'], marker='x')
ax[0].set_yticks(range(num_bandits))
ax[0].set_xlabel("Iteration Number")
ax[0].set_ylabel("Selected Bandit")
ax[0].set_title("Bandit chosen at each iteration (Thompson Sampling)")

ax[1].scatter(range(num_iterations), UCB_round_stats['bandit'], marker='x')
ax[1].set_yticks(range(num_bandits))
ax[1].set_xlabel("Iteration Number")
ax[1].set_ylabel("Selected Bandit")
ax[1].set_title("Bandit chosen at each iteration (UCB)")

max_mean_index = np.argmax(TS_bandits.bandits)
for i in range(num_bandits):
    if i == max_mean_index:
        ax[1].text(x=num_iterations+700, y=i-0.2, s="$\mu_{}$ = {}".format(i, TS_bandits.bandits[i]), color="red")
    else:
        ax[1].text(x=num_iterations+700, y=i-0.2, s="$\mu_{}$ = {}".format(i, TS_bandits.bandits[i]))

In [ ]:
# Plot the accumulated regret through the experiment
fig, ax = plt.subplots(1, 2, figsize=(20,5))
ax[0].plot(range(num_iterations), TS_round_stats['regret'])
ax[0].set_xlabel("Iteration Number")
ax[0].set_ylabel("Cumulative Regret")
ax[0].set_title("Total Regret (Thompson Sampling)")

ax[1].plot(range(num_iterations), UCB_round_stats['regret'])
ax[1].set_xlabel("Iteration Number")
ax[1].set_ylabel("Cumulative Regret")
ax[1].set_title("Total Regret (UCB)")